In [1]:
!kaggle datasets download -d  harishvutukuri/dogs-vs-wolves -p data

dogs-vs-wolves.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
!unzip -q -u data/dogs-vs-wolves.zip -d data/dogs-vs-wolves

In [3]:
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile # TF temporal bugfix

In [4]:
%load_ext tensorboard

## Imports

In [5]:
import os

# Torch #
import torch

import torchero
from torchero.models.vision import BinaryImageClassificationModel # Training the model
from torchero.callbacks import ProgbarLogger # Model supervising
from torchero.callbacks.tensorboard import TensorBoardLogger # Tensorboard metrics training
from torchero.utils.data import train_test_split
from torchero.utils.vision import show_imagegrid_dataset, transforms, datasets
from torchero.utils.tensorboard import SummaryWriter, write_imagegrid_dataset, write_model
from matplotlib import pyplot as plt

In [6]:
!rm -rf runs/

In [7]:
train_writer = SummaryWriter('runs/Dogs vs Wolves/Train')
val_writer = SummaryWriter('runs/Dogs vs Wolves/Test')

## Training dataset

In [8]:
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor()])

In [9]:
dataset = datasets.ImageFolder(root='data/dogs-vs-wolves/data', transform=transform)

In [10]:
train_ds, val_ds = train_test_split(dataset)

In [11]:
write_imagegrid_dataset(train_writer, 'Dataset', train_ds)

In [12]:
write_imagegrid_dataset(val_writer, 'Dataset', val_ds)

In [13]:
%tensorboard --logdir runs/

## Define model

In [14]:
model = BinaryImageClassificationModel.from_pretrained('resnet18', num_outputs=1, transform=transform)
model.labels = ['is_wolf']

In [15]:
write_model(train_writer, model.model, train_ds[0][0].unsqueeze(0))

/usr/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1153.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


for param_group in trainer.optimizer.param_groups:
    print(param_group["lr"])

## Train model

In [16]:
model.compile(optimizer='adam',
              callbacks=[ProgbarLogger(notebook=True),
                         TensorBoardLogger(train_writer, test_writer=val_writer)]).cuda()

In [17]:
history = model.fit(train_ds,
          val_ds,
          epochs=10,
          batch_size=64)

  0%|          | 0/10 [00:00<?, ?epoch/s]

  0%|          | 0/25 [00:00<?, ? batchs/s]

  0%|          | 0/25 [00:00<?, ? batchs/s]

  0%|          | 0/25 [00:00<?, ? batchs/s]

  0%|          | 0/25 [00:00<?, ? batchs/s]

...

  0%|          | 0/25 [00:00<?, ? batchs/s]

  0%|          | 0/25 [00:00<?, ? batchs/s]

  0%|          | 0/25 [00:00<?, ? batchs/s]

  0%|          | 0/25 [00:00<?, ? batchs/s]

  0%|          | 0/25 [00:00<?, ? batchs/s]

### Writting Embeddings to Projector

In [18]:
summary_writer = SummaryWriter('runs/Dogs vs Wolves')

In [19]:
from torch.utils.data import DataLoader
from itertools import chain, islice

def embed(dl, model, thumb_fn, split=None):
    for ims, labels in dl:
        with torch.no_grad():
            embs, output = model.model.embed(ims.cuda())
        probas =  torch.nn.functional.sigmoid(output).squeeze(-1)
        for im, label, emb, probs in zip(ims, labels, embs, probas):
            yield thumb_fn(im), emb.cpu(), probs.item(), label.item(), (probs>0.5).item(), ((probs>.5)==label).item(), split

train_dl = DataLoader(train_ds, batch_size=10, shuffle=True)
test_dl = DataLoader(val_ds, batch_size=10, shuffle=True)

thumb = transforms.Compose([transforms.ToPILImage(), transforms.Resize((64, 64)), transforms.ToTensor()])
embs = chain(islice(embed(train_dl, model, thumb, 'train'), 100),
             islice(embed(test_dl, model, thumb, 'test'), 100))
images, vectors, probas, gt_labels, pred_labels, correct, split = zip(*embs)
images = torch.stack(images)
vectors = torch.stack(vectors)

summary_writer.add_embedding(vectors,
                             metadata=list(zip(gt_labels, split, pred_labels, probas, correct)),
                             label_img=images,
                             global_step=0,
                             tag='embeddings',
                             metadata_header=['label', 'mode', 'pred_label', 'proba', 'correct'])

/usr/lib/python3.9/site-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [20]:
summary_writer.flush()

In [21]:
summary_writer.close()